In [6]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
import csv
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from pandas import DataFrame
from multipledispatch import dispatch

class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        #불용어사전 추가중
        with open('C:/Users/user/stopwords.txt', 'r', encoding='utf-8') as f:
            list_file = f.readlines()
        self.stopwords = list_file[0].split(",")
        
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)      
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences

    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence)) 
                                       if noun not in self.stopwords and len(noun) > 1]))
        return nouns


class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

#TextRank
class Rank(object):
    def get_ranks(self, graph, d=0.85): 
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 
            link_sum = np.sum(A[:,id])
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) 
        return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        
        self.graph_matrix = GraphMatrix()
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)

        self.rank = Rank()
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    
    def keywords(self, word_num=5):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

        keywords = []
        index=[]
        
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)

        for idx in index:
            keywords.append(self.idx2word[idx])

        return keywords

data = pd.read_csv('통합_keyword.csv', low_memory=False)

#global key
def get_keys(*args):
    cnt = 0
    row =[]
    key_list = []
    del_list = []
    key = str('')
    web_name = str('')
    
    if(len(args) <= 8):
        for k in args:
            key += data.iloc[data.index[data['제목'] == args[cnt]][0], 8]
            #key_list.append(data.index[data['제목'] == args[cnt]][0])
            for s in data.index[data['제목'] == args[cnt]].values:
                key_list.append(s)
            web_name += (k+'  ')
            cnt += 1
    else:
        for k in range(8):
            key += data.iloc[data.index[data['제목'] == args[cnt]][0], 8]
            #key_list.append(data.index[data['제목'] == args[cnt]][0])
            for s in data.index[data['제목'] == args[cnt]].values:
                key_list.append(s)
            web_name += (args[cnt]+'  ')
            cnt += 1
        print("최대 8개 작품까지만 등록 가능합니다.")
    key_args = 2*(len(args)-1) + 5
    print(web_name + '\n')
    textrank = TextRank(key)
    text_key = textrank.keywords(key_args)
    
    row = [len(data), '추천키워드','작가','장르','줄거리','url','썸네일','플랫폼',str(text_key)]
    df = data.append(pd.Series(row, index=data.columns),ignore_index=True)
    df.iloc[-1] = row

    #print(row[8]) #추천키워드 목록
    
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['키워드'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    index = len(df)-1
    sim = sorted(list(enumerate(cosine_sim[index])), key=lambda x: x[1], reverse=True)
    key_list.append(index)

    #입력한 작품 제외
    for a in key_list:
        for j in range(len(sim)):
            if(sim[j][0] == a): 
                del_list.append(j)

    #입력한 작품 제외
    for idx in sorted(del_list, reverse = True):
        del sim[idx]


    #추천키워드와 코사인 유사도가 높은 7개 작품 출력
    web_indices = [index[0] for index in sim[0:7]]
    
    for indx in web_indices:
        print(df.iloc[indx, 1])

In [7]:
get_keys('부마님 거기 있어줄래요')
print("\n")
get_keys('방과 후 전쟁활동', '데드데이즈(DEAD DAYS)')
print("\n")
get_keys('부마님 거기 있어줄래요', '공주님 마음대로!', '너와 사는 오늘')
print("\n")
get_keys('방과 후 전쟁활동', '데드데이즈(DEAD DAYS)','하이브 3','정글쥬스')
print("\n")
get_keys('부마님 거기 있어줄래요', '공주님 마음대로!', '너와 사는 오늘', '우리 내일 이혼해요', 'N번째 연애', '유니크한 그녀', '방과 후 전쟁활동', '심연의 하늘 시즌4')
print("\n")
get_keys('방과 후 전쟁활동', '데드데이즈(DEAD DAYS)','하이브 3','정글쥬스','부마님 거기 있어줄래요', '공주님 마음대로!', '너와 사는 오늘', '우리 내일 이혼해요', 'N번째 연애', '유니크한 그녀', '심연의 하늘 시즌4', '연애혁명' , '신도림', '하이브 1~2')

부마님 거기 있어줄래요  

공주전쟁 [연재]
공주전쟁
유니크한 그녀
연애혁명
천일야화
여혜
관리 마츠리카 전기 [연재]


방과 후 전쟁활동  데드데이즈(DEAD DAYS)  

리턴 서바이벌
공부하기 좋은 날
리즌
의도적 외면
심연의 하늘 시즌4
개미
사람의 조각


부마님 거기 있어줄래요  공주님 마음대로!  너와 사는 오늘  

솔로가 하는 연애
당신만 몰라!
대놓고 사내연애
그녀는 흡!혈귀
헤어진 다음날
파멸 엔딩뿐인 악역영애, 맛있는 요리를 만들며 지내겠습니다. [연재]
월식(Lunar Eclipse)


방과 후 전쟁활동  데드데이즈(DEAD DAYS)  하이브 3  정글쥬스  

하이브 1~2
신도림
심연의 하늘 시즌4
캉타우
스퍼맨 시즌1
트레이스
개장수


부마님 거기 있어줄래요  공주님 마음대로!  너와 사는 오늘  우리 내일 이혼해요  N번째 연애  유니크한 그녀  방과 후 전쟁활동  심연의 하늘 시즌4  

권태
당신의 후회는 받지 않겠습니다
우연일까?
그녀는 흡!혈귀
데드데이즈(DEAD DAYS)
그대 마음 한 스푼
죽고 못사는 연애


최대 8개 작품까지만 등록 가능합니다.
방과 후 전쟁활동  데드데이즈(DEAD DAYS)  하이브 3  정글쥬스  부마님 거기 있어줄래요  공주님 마음대로!  너와 사는 오늘  우리 내일 이혼해요  

개장수
심연의 하늘 시즌4
당신의 후회는 받지 않겠습니다
하이브 1~2
우연일까?
그녀는 흡!혈귀
스퍼맨 시즌1
